In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Part1: Calculation for EC145 Data

### 1. Defining Parameters 

In [3]:
rho = 1.225                        #Density at sea level
weight = 3300                      #Weight of helicopter

#rotor parameters
blades = 4                         #Number of blades
rpm = 383.36                       #Rotor RPM   
omega = 2*np.pi*rpm/60

C = 0.312                          #Chord length
R = 5.5                            #Radius



#Tail Rotor Parameters
tail_COG = 7.1
tail_radius = 0.981
tail_chors = 0.12
tail_solidity = (2*tail_chors)/(np.pi*tail_radius)
tail_omega= 124.6

a=5.8                              #coefficient of lift
Cd_0 = 0.010

In [5]:
#Thrust Calculation
Thrust = weight*9.81            


#Flat Plate area
plate_area = 1.95096


#solidity
Solidity = (blades*C)/(np.pi*R)

#Power Factor 
k=1.14

#Installation loss
f=0.97


### 2.  Absolute and Service Ceiling calulations

[Power Data at Page Number 8](https://exclusiveaircraft.co.uk/sites/default/files/brochure/Eurocopter-EC145-Brochure.pdf)

In [11]:
#For twin engine Power Parameters
max_cont_power = 632*1000         #in kW


In [12]:
#Calculating the Tail rotor power(includes tail induced and profile power)
def Tail_rotor_power(h, Thrust):
    rho_ratio = ((1-(0.00198*h/288.16))**4.2553)
    Torque = Power_h_required_service(h,Thrust)/omega
    tail_thrust = Torque/tail_COG
    tail_induced_power = tail_thrust*np.sqrt(tail_thrust/(2*rho*rho_ratio*np.pi*tail_radius*tail_radius))
    tail_profile_power = (tail_solidity*Cd_0*rho_ratio*rho*(tail_omega**3)*(tail_radius**5)*np.pi)/8
    tail_power = tail_induced_power+tail_profile_power
    return tail_power

In [13]:
#Calculate the power available at height h
def Power_h_available(h,power):
    ratio = ((1-(0.00198*h/288.16))**4.2553)
    power_at_h = power*ratio*f
    return power_at_h

In [14]:
#Power required by main rotor including induced, profile power
def Power_h_required_service(h,Thrust):
    rho_ratio = ((1-(0.00198*h/288.16))**4.2553)
    induced_power = (Thrust*np.sqrt(Thrust/(2*rho*rho_ratio*np.pi*R*R)))/1.06
    profile_power = (Solidity*Cd_0*rho_ratio*rho*(omega**3)*(R**5)*np.pi)/8
    
    power = induced_power + profile_power 
    #print(power)
    return power

In [15]:
#Max climb rate at height h for given max power and Thrust
def max_climb_rate(h,power_installed, Thrust):
    power_available = Power_h_available(h,power_installed)
    power_required = Power_h_required_service(h, Thrust)+Tail_rotor_power(h,Thrust)
    climb_rate = (power_available-power_required)/(9.81*weight)
    return climb_rate
    

In [16]:
height = np.arange(4700,6000,50)
for i in range(len(height)):
    max_climbrate = max_climb_rate(height[i],max_cont_power,Thrust )
    print(height[i], max_climbrate)

4700 0.6290161145240898
4750 0.598728152963249
4800 0.5684509101083103
4850 0.5381843459376742
4900 0.5079284203912867
4950 0.4776830933705174
5000 0.4474483247380186
5050 0.41722407431765546
5100 0.38701030189431557
5150 0.3568069672138296
5200 0.3266140299828024
5250 0.29643144986853925
5300 0.2662591864988355
5350 0.23609719946194413
5400 0.20594544830638176
5450 0.17580389254083134
5500 0.14567249163398405
5550 0.11555120501443852
5600 0.08543999207056971
5650 0.05533881215033816
5700 0.02524762456126493
5750 -0.004833611429791402
5800 -0.034904936596719234
5850 -0.06496639175430859
5900 -0.09501801775836437
5950 -0.1250598555058466


In [17]:
print("Absolute Ceiling:",5720)
print("service Ceiling:",4900)

Absolute Ceiling: 5720
service Ceiling: 4900


### 3.  Maximum forward speed from power considerations

In [18]:
#Max forward speed calculation 
#Includes the required power due main, tail rotor and parasite drag
#Max speed is when power required and power available are equal
def max_forward_speed(h, Thrust):
    rho_ratio = ((1-(0.00198*h/288.16))**4.2553)
    power_available = Power_h_available(h,max_cont_power)
    power_required = Power_h_required_service(h, Thrust) + Tail_rotor_power(h,Thrust) 
    
    max_forward_speed = (((power_available - power_required)*2)/(rho*rho_ratio*plate_area*0.8))**(1/3)
    return max_forward_speed

In [19]:
print("Max Forward Velocity: ", max_forward_speed(420, Thrust))

Max Forward Velocity:  48.196542270474026


### 4. Range and endurance for a typical operating condition


In [20]:
#Fuel Parameters
total_fuel = 694                 #in kG
cruise_speed = 67.5              #in m/s

fuel_consumption_cruise = 234/(60*60)          #in Kg/s
fuel_consumption_65_kias = 197/(60*60)         #in Kg/s

permissable_TOF = total_fuel/fuel_consumption_cruise

In [21]:
#power required at recommended cruise speed which is 67.5
#we will calculate at h=420m
def Power_at_recommended_cruise():
    rho_h = 1.21
    Drag = 0.5*rho_h*plate_area*(cruise_speed**2)*0.8
    alpha_tpp = np.arctan(Drag/(weight*9.81))
    Thrust_for_R = (weight*9.81)/(np.cos(alpha_tpp))
    mu = cruise_speed*np.cos(alpha_tpp)/(omega*R)
    induced_power = k*Thrust_for_R**2/(2*mu*rho_h*np.pi*(R**3)*omega)
    parasite_power = Drag*cruise_speed
    profile_power = (Solidity*Cd_0*rho_h*(omega**3)*(R**5)*np.pi*(1+4.6*(mu**2)))/8
    Total_power_for_R_cruise = induced_power+parasite_power+profile_power
    return Total_power_for_R_cruise

In [70]:
234/(60*60)

0.065

In [22]:
rotor_power_for_R_cruise = Power_at_recommended_cruise()
rotor_power_for_R_cruise

528380.4102881963

In [23]:
#Similarly for tail rotor
def Tail_power_at_recommended_cruise():
    rho_h = 1.21
    Torque = rotor_power_for_R_cruise/omega
    tail_thrust = Torque/tail_COG
    tail_induces_power = tail_thrust*np.sqrt(tail_thrust/(2*rho_h*np.pi*tail_radius*tail_radius))
    tail_profile_power = (tail_solidity*Cd_0*rho_h*(tail_omega**3)*(tail_radius**5)*np.pi)/8
    tail_power = tail_induces_power+tail_profile_power
    return tail_power

In [24]:
Tail_power_at_recommended_cruise()

30157.520212978277

In [25]:
Total_power_required_for_recommended_cruise = Power_at_recommended_cruise()+Tail_power_at_recommended_cruise()


In [26]:
fuel_per_Watt = fuel_consumption_cruise/Total_power_required_for_recommended_cruise

In [54]:
print("fuel_per_Watt:" ,fuel_per_Watt)        
#We will use this for our design beacuse we dont know fuel consumption required for recommended cruise speed, 
#so we will take that from here

fuel_per_Watt: 1.1637526558254636e-07


In [29]:
def Range():
    range_ = cruise_speed*permissable_TOF
    return range_

In [30]:
print("Range: ", Range())

Range:  720692.3076923076


In [31]:
def Endurance():
    endurance = total_fuel/fuel_consumption_65_kias
    return endurance

In [32]:
print("Endurance: ", Endurance())

Endurance:  12682.23350253807


### 5. Autorotative index

In [69]:
#defining inertia parameters
rotor_weight = 36.3                                         #one ROTOR weight
J_rotor_blades = (blades*(4*R*R + C*C)*rotor_weight)/12     #inertia of all rotors
tail_rotor_weight = 2.49                                    #in Kg
J_tail_rotors = 2*((4*(tail_radius**2)+(tail_chors**2)))*tail_rotor_weight/12
J_total = J_rotor_blades + J_tail_rotors*tail_omega/omega

#Lock Number
lock_number = (rho*C*a*(R**4))/J_total
print("Lock Number:", lock_number)

Lock Number: 1.3134434385010927


In [34]:
def AI():
    rho_h = 1.21
    Drag = 0.5*rho_h*plate_area*(cruise_speed**2)*0.8
    alpha_tpp = np.arctan(Drag/(weight*9.81))
    Thrust = (weight*9.81)/(np.cos(alpha_tpp))
    autorotative_index = (J_total*(omega**2)*rho_h*np.pi*(R**2))/(weight*rho*Thrust)
    return autorotative_index

In [36]:
print("Autorotative Index: ",AI())

Autorotative Index:  2.0639030343987788


# Part2: Design Parameters and Calculation for new Helicopter

 1. Absolute ceiling for previous case: 5720
 2. Now Required Absolute ceiling: 8008

In [37]:
#Density as new absolue ceiling
rho_new = rho_ratio = ((1-(0.00198*8008/288.16))**4.2553)*1.225

In [38]:
#New design Parameters:
#After trying many different variations of quatities we arrive at
weight_new = 3100 
C = 0.297


In [39]:
#New Thrust
Thrust_new = weight_new*9.81

#New solidity
Solidity_new = blades*C/(np.pi*R)

####  Prove of absolute ceiling

In [40]:
Power_avail_new = Power_h_available(8008, max_cont_power)

In [43]:
#Power required after including variation of solidity or Chord length
def Power_required_new_at_H(h,Thrust, solidity):
    rho_ratio = ((1-(0.00198*h/288.16))**4.2553)
    induced_power = (Thrust*np.sqrt(Thrust/(2*rho*rho_ratio*np.pi*R*R)))/1.06
    profile_power = (solidity*Cd_0*rho_ratio*rho*(omega**3)*(R**5)*np.pi)/8
    power = induced_power + profile_power
    return power


Power_required_new = Power_required_new_at_H(8008, Thrust_new,Solidity_new)+Tail_rotor_power(8008,Thrust_new)

In [44]:
print("Power_avail_new: ",Power_avail_new)
print("Power_required_new: ", Power_required_new)
print("Max climb rate: ", (Power_avail_new-Power_required_new)/(9.81*weight_new))

Power_avail_new:  481832.41035964363
Power_required_new:  481771.0752542534
Max climb rate:  0.002016872361653011


Comment: Above we can see that Max climb rate at 1.4 of prevoius absolute ceiling is zero.
Which is what desired.

## Caculation of other Quantities

### 1. Service ceiling

In [45]:
height = np.arange(7000,8000,50)
for i in range(len(height)):
    Power_avail_diff_h = Power_h_available(height[i], max_cont_power)
    Power_required_diff_h = Power_required_new_at_H(height[i], Thrust_new,Solidity_new)+Tail_rotor_power(height[i],Thrust_new)
    max_climbrate = (Power_avail_diff_h-Power_required_diff_h)/(9.81*weight_new)
    print(height[i], max_climbrate)

7000 0.6229663013375233
7050 0.5920614266115735
7100 0.5611676449709082
7150 0.5302849141748017
7200 0.49941319194119443
7250 0.4685524359465289
7300 0.43770260382564596
7350 0.4068636531715917
7400 0.37603554153553886
7450 0.34521822642662425
7500 0.31441166531179165
7550 0.2836158156156945
7600 0.2528306347205058
7650 0.22205607996582125
7700 0.1912921086484826
7750 0.16053867802246113
7800 0.12979574529872367
7850 0.09906326764504679
7900 0.06834120218593279
7950 0.03762950600242578


In [46]:
print("Service Ceiling for Our Design(m): ", 7200)

Service Ceiling for Our Design(m):  7200


### 2. Maximum Forward Speed

In [47]:
def max_forward_speed_new(h, Thrust,solidity):
    rho_ratio = ((1-(0.00198*h/288.16))**4.2553)
    power_available = Power_h_available(h,max_cont_power)
    power_required = Power_required_new_at_H(h, Thrust,solidity) + Tail_rotor_power(h,Thrust) 
    
    max_forward_speed = (((power_available - power_required)*2)/(rho*rho_ratio*plate_area*0.8))**(1/3)
    return max_forward_speed

In [48]:
print("Max Forward Velocity for our design: ", max_forward_speed_new(420, Thrust_new,Solidity_new))

Max Forward Velocity for our design:  53.679176997370256


### 3. Range and Endurance

1. We will calculate the Range and endurance at recommended cruise speed

In [55]:
#Fuel Parameters
total_fuel = 694                 #in kG
cruise_speed = 67.5              #in m/s

fuel_consumption_cruise = 234/(60*60)          #in Kg/s
fuel_consumption_65_kias = 197/(60*60)         #in Kg/s

permissable_TOF = total_fuel/fuel_consumption_cruise

In [56]:
#power required at recommended cruise speed
#we will calculate at h=420m
def Power_at_recommended_cruise_new():
    rho_h = 1.21
    Drag = 0.5*rho_h*plate_area*(cruise_speed**2)*0.8
    alpha_tpp = np.arctan(Drag/(weight_new*9.81))
    Thrust_for_R = (weight_new*9.81)/(np.cos(alpha_tpp))
    mu = cruise_speed*np.cos(alpha_tpp)/(omega*R)
    induced_power = k*Thrust_for_R**2/(2*mu*rho_h*np.pi*(R**3)*omega)
    parasite_power = Drag*cruise_speed
    profile_power = (Solidity_new*Cd_0*rho_h*(omega**3)*(R**5)*np.pi*(1+4.6*(mu**2)))/8
    Total_power_for_R_cruise = induced_power+parasite_power+profile_power
    return Total_power_for_R_cruise

In [57]:
rotor_power_for_R_cruise_new = Power_at_recommended_cruise_new()

In [58]:
#Similarly for tail rotor
def Tail_power_at_recommended_cruise_new():
    rho_h = 1.21
    Torque = rotor_power_for_R_cruise_new/omega
    tail_thrust = Torque/tail_COG
    tail_induces_power = tail_thrust*np.sqrt(tail_thrust/(2*rho_h*np.pi*tail_radius*tail_radius))
    tail_profile_power = (tail_solidity*Cd_0*rho_h*(tail_omega**3)*(tail_radius**5)*np.pi)/8
    tail_power = tail_induces_power+tail_profile_power
    return tail_power

In [59]:
Total_power_required_for_recommended_cruise_new = Power_at_recommended_cruise_new()+Tail_power_at_recommended_cruise_new()

In [60]:
fuel_consumption_cruise_new = fuel_per_Watt*Total_power_required_for_recommended_cruise_new

In [61]:
fuel_consumption_cruise_new

0.06288356076093192

In [62]:
def Endurance_new():
    endurance = total_fuel/fuel_consumption_cruise_new
    return endurance

In [63]:
Endurance_new_design = Endurance_new()

In [64]:
def Range_new():
    range_ = Endurance_new_design*cruise_speed
    return range_

In [65]:
print("Endurance for our design:", Endurance_new())
print('Range for our design: ', Range_new())

Endurance for our design: 11036.270713715148
Range for our design:  744948.2731757725


### 4. Autorotative Index

In [66]:
#defining inertia parameters
C_new = 0.297
rotor_weight = 36.3                                         #one ROTOR weight
J_rotor_blades = (blades*(4*R*R + C_new*C_new)*rotor_weight)/12     #inertia of all rotors
tail_rotor_weight = 2.49                                    #in Kg
J_tail_rotors = 2*((4*(tail_radius**2)+(tail_chors**2)))*tail_rotor_weight/12
J_total = J_rotor_blades + J_tail_rotors*tail_omega/omega

In [67]:
def AI_new():
    rho_h = 1.21
    Drag = 0.5*rho_h*plate_area*(cruise_speed**2)*0.8
    alpha_tpp = np.arctan(Drag/(weight_new*9.81))
    Thrust = (weight_new*9.81)/(np.cos(alpha_tpp))
    autorotative_index = (J_total*(omega**2)*rho_h*np.pi*(R**2))/(weight_new*rho*Thrust)
    return autorotative_index

In [68]:
print("Autorotative Index for our design : ",AI_new())

Autorotative Index for our design :  2.3359295671775735
